<a href="https://colab.research.google.com/github/sum-coderepo/DeepLearning-Pytorch/blob/master/PytorchTutorials/Ful_conNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
model = NN(784, 10)
x = torch.randn(64, 784)
print(model.forward(x).shape)

torch.Size([64, 10])


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
input_size = 784
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 1

In [12]:
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [16]:
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [17]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [18]:
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network

In [19]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and optimizer

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train Network

In [22]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

In [23]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [24]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 56030 / 60000 with accuracy 93.38
Checking accuracy on test data
Got 9340 / 10000 with accuracy 93.40


# Predicting Insurance price

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [44]:
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [127]:
train_data = pd.read_csv("dataset/insurance.csv")
train_data = pd.get_dummies(train_data, columns=["sex", "smoker", "region"])
cols = ["age", "bmi", "children",  "sex_female", "sex_male", "smoker_no", "smoker_yes","region_northeast",  "region_northwest", "region_southeast", "region_southwest", "charges"]
train_data = train_data.reindex(columns=cols)
train_data.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,charges
0,19,27.900,0,1,0,0,1,0,0,0,1,16884.92400
1,18,33.770,1,0,1,1,0,0,0,1,0,1725.55230
2,28,33.000,3,0,1,1,0,0,0,1,0,4449.46200
3,33,22.705,0,0,1,1,0,0,1,0,0,21984.47061
4,32,28.880,0,0,1,1,0,0,1,0,0,3866.85520


In [128]:
scaler = MinMaxScaler()

In [129]:
X_train = train_data.iloc[:,0:-1].to_numpy()
X_train = scaler.fit_transform(X_train)

In [130]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(train_data.iloc[:,-1].to_numpy())

In [131]:
X_train

tensor([[0.0217, 0.3212, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.0000, 0.4791, 0.2000,  ..., 0.0000, 1.0000, 0.0000],
        [0.2174, 0.4584, 0.6000,  ..., 0.0000, 1.0000, 0.0000],
        ...,
        [0.0000, 0.5620, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.0652, 0.2647, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.9348, 0.3527, 0.0000,  ..., 1.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [132]:
X_train.dtype, y_train.dtype

(torch.float64, torch.float64)

In [142]:
class LinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(11, 1)  # One in and one out

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [143]:
input_size = 11
output_size = 1

In [144]:
model = LinearRegressionModel()

In [145]:
learning_rate = 0.01
l = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr =learning_rate )

In [146]:
num_epochs = 10000
for epoch in range(num_epochs):
    #forward feed
    y_pred = model(X_train.float().requires_grad_())

    #calculate the loss
    loss= l(y_pred.float(), y_train.float())

    #backward propagation: calculate gradients
    loss.backward()

    #update the weights
    optimizer.step()

    #clear out the gradients from the last step loss.backward()
    optimizer.zero_grad()
    
    print('epoch {}, loss {}'.format(epoch, loss.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1338])) that is different to the input size (torch.Size([1338, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
epoch 5000, loss 146543856.0
epoch 5001, loss 146543856.0
epoch 5002, loss 146543856.0
epoch 5003, loss 146543856.0
epoch 5004, loss 146543856.0
epoch 5005, loss 146543856.0
epoch 5006, loss 146543856.0
epoch 5007, loss 146543856.0
epoch 5008, loss 146543856.0
epoch 5009, loss 146543856.0
epoch 5010, loss 146543856.0
epoch 5011, loss 146543856.0
epoch 5012, loss 146543856.0
epoch 5013, loss 146543856.0
epoch 5014, loss 146543856.0
epoch 5015, loss 146543856.0
epoch 5016, loss 146543856.0
epoch 5017, loss 146543840.0
epoch 5018, loss 146543856.0
epoch 5019, loss 146543840.0
epoch 5020, loss 146543856.0
epoch 5021, loss 146543840.0
epoch 5022, loss 146543856.0
epoch 5023, loss 146543856.0
epoch 5024, loss 146543840.0
epoch 5025, loss 146543840.0
epoch 5026, loss 146543840.0
epoch 5027, loss 146543840.0
epoch 5028, loss 146543840.0
epoch 5029, loss 146543840.0
epoch 5030, loss 146543840.0
epoch 5031, loss 146543840.0
epoch 5032, loss 1465